In [1]:
import tensorflow as tf

In [2]:
!wget --no-check-certificate \
  ftp://cs.stanford.edu/cs/cvgl/Stanford_Online_Products.zip \
  -O /tmp/Stanford_Online_Products.zip

--2021-09-29 23:16:55--  ftp://cs.stanford.edu/cs/cvgl/Stanford_Online_Products.zip
           => ‘/tmp/Stanford_Online_Products.zip’
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /cs/cvgl ... done.
==> SIZE Stanford_Online_Products.zip ... 3083860082
==> PASV ... done.    ==> RETR Stanford_Online_Products.zip ... done.
Length: 3083860082 (2.9G) (unauthoritative)

Stanford_Online_Pro 100%[===================>]   2.87G  25.9MB/s    in 92s     

2021-09-29 23:18:27 (32.1 MB/s) - ‘/tmp/Stanford_Online_Products.zip’ saved [3083860082]



In [3]:
import zipfile, os

file_zip = '/tmp/Stanford_Online_Products.zip'
zip_ref = zipfile.ZipFile(file_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

base_dir = '/content/Stanford_Online_Products'

In [4]:
os.listdir(base_dir)

['Ebay_final.txt',
 'kettle_final.txt',
 'table_final',
 'Ebay_info.txt',
 'Ebay_train.txt',
 'toaster_final.txt',
 'sofa_final.txt',
 'mug_final',
 'table_final.txt',
 'bicycle_final.txt',
 'chair_final',
 'Ebay_test.txt',
 'README',
 'LICENSE',
 'bicycle_final',
 'fan_final',
 'chair_final.txt',
 'mug_final.txt',
 'coffee_maker_final.txt',
 'cabinet_final.txt',
 'lamp_final',
 'toaster_final',
 'sofa_final',
 'lamp_final.txt',
 'coffee_maker_final',
 'kettle_final',
 'cabinet_final',
 'stapler_final',
 'fan_final.txt',
 'stapler_final.txt']

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale = 1./225,
                    rotation_range = 20,
                    horizontal_flip = True,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split = 0.2
)

In [6]:
# deklarasi variabel global ukuran gambar
image_size = (224, 224)

train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size = image_size,
        classes=['lamp_final', 'mug_final', 'sofa_final'],
        batch_size = 128,
        class_mode = 'categorical',
        subset='training'
)
 
val_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size = image_size,
        classes=['lamp_final', 'mug_final', 'sofa_final'],
        batch_size = 64,
        class_mode = 'categorical',
        subset='validation'
)

Found 26870 images belonging to 3 classes.
Found 6717 images belonging to 3 classes.


In [7]:
# membuat class callback untuk stopping
class myCallback(tf.keras.callbacks.Callback):
   def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy') >= 0.92 and logs.get('val_accuracy') >= 0.92):
          print("\nAkurasi telah mencapai >=92%!")
          self.model.stop_training = True

callbacks = myCallback()

In [8]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))) 
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(3, activation = "softmax"))

In [9]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
# deklarasi variabel global epoch
epochs = 500

# membuat objek callback untuk early stopping dan reduksi learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=15, verbose=2, min_lr=0.00001)

# latih model dengan model.fit
history = model.fit(train_generator,
                    epochs = epochs, 
                    validation_data = val_generator,
                    verbose = 2, 
                    steps_per_epoch = 25,
                    callbacks=[callbacks, reduce_lr],
                    validation_steps = 25)


Epoch 1/500
25/25 - 102s - loss: 1.4016 - accuracy: 0.3847 - val_loss: 1.0774 - val_accuracy: 0.3925
Epoch 2/500
25/25 - 65s - loss: 1.0416 - accuracy: 0.4047 - val_loss: 1.0281 - val_accuracy: 0.4544
Epoch 3/500
25/25 - 65s - loss: 1.0038 - accuracy: 0.4509 - val_loss: 1.0328 - val_accuracy: 0.4894
Epoch 4/500
25/25 - 65s - loss: 0.9666 - accuracy: 0.4994 - val_loss: 0.9693 - val_accuracy: 0.5106
Epoch 5/500
25/25 - 68s - loss: 0.9081 - accuracy: 0.5378 - val_loss: 0.9585 - val_accuracy: 0.5125
Epoch 6/500
25/25 - 66s - loss: 0.9218 - accuracy: 0.5391 - val_loss: 0.9036 - val_accuracy: 0.5838
Epoch 7/500
25/25 - 66s - loss: 0.8728 - accuracy: 0.5738 - val_loss: 0.8860 - val_accuracy: 0.5669
Epoch 8/500
25/25 - 66s - loss: 0.8417 - accuracy: 0.6041 - val_loss: 0.8527 - val_accuracy: 0.6069
Epoch 9/500
25/25 - 67s - loss: 0.8257 - accuracy: 0.6331 - val_loss: 0.8279 - val_accuracy: 0.6231
Epoch 10/500
25/25 - 67s - loss: 0.7946 - accuracy: 0.6447 - val_loss: 0.8060 - val_accuracy: 0.627

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)